In [1]:
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader, random_split
from models_utils.GLOBALS import *
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

In [3]:
# get train data
data = pd.read_csv('../csv/secret_all_data_features.csv')
data['sensor'] = data['sensor'].map(sensor_mapping)
data.drop('sample_id', axis=1, inplace=True)

In [4]:
# LSTM model to continue the sequence
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the last time step
        return out


class SequenceDataset(Dataset):
    def __init__(self, features, labels=None, seq_length=10):
        self.features = features
        self.labels = labels
        self.seq_length = seq_length

    def __len__(self):
        return len(self.features) - self.seq_length

    def __getitem__(self, idx):
        idx_end = idx + self.seq_length
        sequence_features = self.features.iloc[idx:idx_end].values
        sequence_features = torch.tensor(sequence_features, dtype=torch.float)
        if self.labels is not None:
            label = self.labels.iloc[idx_end] if self.labels is not None else None
            label = torch.tensor(label, dtype=torch.long)
            return sequence_features, label
        return sequence_features

In [5]:
# prepare data
sequence_length = 500
train_data = data[:77304].copy()
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
features_columns = data.columns.drop('activity').tolist()

In [6]:
# normalize data
scaler = MinMaxScaler()
train_data_normalized = pd.DataFrame(scaler.fit_transform(train_data.drop('activity', axis=1)),
                                     columns=train_data.columns[:-1])
train_data_normalized['activity'] = train_data['activity']
train_data = train_data_normalized

In [9]:
# train LSTM model
train_dataset = SequenceDataset(train_data[features_columns], train_data['activity'], sequence_length)
total_size = len(train_dataset)
train_size = int(total_size * 0.8)
val_size = total_size - train_size  


train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True) 
val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

best_loss = float('inf') 
best_model_path = '../models/best_model_LSTM_features_cnn5.pth' 

# model
input_dim = len(features_columns)  
hidden_dim = 128  
layer_dim = 2  
output_dim = len(train_data['activity'].unique())  
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2, verbose=True)


num_epochs = 300
for epoch in range(num_epochs):
    print('---------------------------------------')
    model.train()  
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # validation 
    model.eval()  
    total_val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:  
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    scheduler.step(avg_val_loss)
    print(f'Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

    # save the best model
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f'Saved new best model at epoch {epoch + 1} with validation loss: {best_loss:.4f}')

---------------------------------------
Epoch 1, Train Loss: 2.3322, Val Loss: 1.9029
Saved new best model at epoch 1 with validation loss: 1.9029
---------------------------------------
Epoch 2, Train Loss: 1.9763, Val Loss: 1.4168
Saved new best model at epoch 2 with validation loss: 1.4168
---------------------------------------
Epoch 3, Train Loss: 1.5397, Val Loss: 1.6181
---------------------------------------
Epoch 4, Train Loss: 0.8439, Val Loss: 1.1371
Saved new best model at epoch 4 with validation loss: 1.1371
---------------------------------------
Epoch 5, Train Loss: 0.5264, Val Loss: 0.5038
Saved new best model at epoch 5 with validation loss: 0.5038
---------------------------------------
Epoch 6, Train Loss: 0.4854, Val Loss: 0.3849
Saved new best model at epoch 6 with validation loss: 0.3849
---------------------------------------
Epoch 7, Train Loss: 0.3322, Val Loss: 0.3543
Saved new best model at epoch 7 with validation loss: 0.3543
--------------------------------

KeyboardInterrupt: 

In [10]:
# load the best model
model.load_state_dict(torch.load(best_model_path))
model = model.to(device)  

In [71]:
# get test data
test_data = data[77304-sequence_length:].copy()
test_data = pd.DataFrame(scaler.transform(test_data.drop('activity', axis=1)), columns=test_data.columns[:-1])

In [72]:
# get and save results for test data
test_dataset = SequenceDataset(test_data[features_columns], seq_length=sequence_length)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
model.eval()  # Set model to evaluation mode
predictions = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to(device)  # Move to device
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
predictions_df = pd.DataFrame(predictions, columns=['prediction'])
predictions_df['activity'] = predictions_df['prediction'].map(id_activity_mapping)

In [97]:
results = pd.read_csv('../csv/secret_all_data_features.csv')[77304:]['sample_id'].rename({'sample_id':'id'}).reset_index(drop=True)
results['activity'] = predictions_df['activity']
results.to_csv('../csv/lstm_results5.csv', index=False)